In [4]:
import numpy as np
import pandas as pd
from sympy import Matrix

In [5]:
def find_tight_rows(_A, _z, _b, epi=1e-8):
    # the following do multiplication of Z with the corresponding A.
    product = np.dot(_A, _z)

    # Check if the product equals b_i for each row. This will also give us a TRUE/FALSE mask of the tight rows
    tight_mask = np.abs(product - _b) < epi #this is done to prevent rounding errors in 3D cases
    # print(tight_mask.shape)
    # print(tight_mask)
    # print(_A.shape)
    A1 = _A[tight_mask.flatten()]
    A2 = _A[~tight_mask.flatten()]
    
#     print("//////")
#     print(_A)
#     print(A1)
#     print(A2)
    return tight_mask, A1, A2

In [13]:
# def check_degeneracy(A_active):
#     return A_active.shape[0] > A_active.shape[1]

In [7]:
def has_degeneracy(self, b):
        # Method to check if the problem has degeneracy

        # Find indices where the constraints are satisfied with equality
        equality_indices = np.where(np.abs(np.dot(self.A, (self.polishing(b))) - b) < self.Tolerance)[0]

        # check if the number of equality indices is equal to the number of variables
        return len(equality_indices) != self.A.shape[1]

def correct_degeneracy(self, b):
    # Method to correct degeneracy in the linear programming problem

    # calculate the number of rows to be modified
    rows_modified = (self.A.shape[0]) - (self.A.shape[1])

    n = 0

    while True:
        n += 1
        temp_b = b.astype(np.float64)

        if n < 1000:
            temp_b[:rows_modified] += np.random.uniform(self.Tolerance, self.Tolerance * 10, size=rows_modified)
        else:
            temp_b[:rows_modified] += np.random.uniform(0.1, 10, size=rows_modified)

        if not self.has_degeneracy(temp_b):
            print('Degeneracy removed')
            return temp_b

In [8]:
def initial_feasible_vertex(A, b, c, z, dim_n):
    
    bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
    
    if len(A_active)==0:
        rank = 0
    else:
        rank = np.linalg.matrix_rank(A_active)
    
    if rank != dim_n:
        
        while rank!=dim_n:
            
            cost = np.dot(c, z)
            print(f"z: {tuple(z.flatten())}, Cost: {np.round(cost[0], 2)}")
            
            direction = None
            
            if A_active.shape[0] == 0:
                direction = np.random.rand(untight_rows.shape[-1])
            else:
                null_space_matrix = null_space(tight_rows)
                direction = null_space_matrix[:, 0]
            
            while True:
                
                for b_, a_ in zip(b[~bool_mask], A_inactive):
                    step_magnitudes.append((b_ - np.dot(a_, z)) / np.dot(a_, direction))
        
                step_magnitudes = np.array(step_magnitudes)
                step_magnitudes = step_magnitudes[np.where(step_magnitudes > 0)[0]]
        
                if len(step_magnitudes) == 0:  #handling unbounded case
                    direction = -1*direction
                else:
                    break
                    
            min_step_magnitude = np.min(step_magnitudes)
            
            z += min_step_magnitude * direction
            
            bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
            
            if len(A_active)==0:
                rank = 0
            else:
                rank = np.linalg.matrix_rank(A_active)
                
                
    return z

In [9]:
def find_optimal_vertex(A, b, c, z):
    
    print("\nJourney to an optimal vertex:")
    
    # Identify active constraints where A·z = b
    bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
    
    # Compute initial reduced costs
    epsilons = -1*np.linalg.inv(A_active).T
    epsilons = epsilons.dot(c)

    while np.any(epsilons > 0):  # While there are negative reduced costs

        
        beta = 1e-1  # Step size
        cost = np.dot(c.T, z)  # Calculate current objective value
        print(f"z: {tuple(z.flatten())}; Cost: {np.round(cost[0], 2)}")

        # Find the direction of improvement (negative reduced cost)
        A_active_inv = np.linalg.inv(A_active)
        directions = -1*A_active_inv
        
        direction = None
        for i, epsilon in enumerate(epsilons):
            if epsilon > 0:
                direction = directions[:, i:i + 1]
                break
        
        if direction is None:
            print("No direction for improvement found.")
            break        
        
        step_magnitudes = []
        # print(A_inactive)
        # print(z)
        # print(b[~bool_mask])
        for b_, a_ in zip(b[~bool_mask], A_inactive):
            step_magnitudes.append((b_ - np.dot(a_, z)) / np.dot(a_, direction))
        
        # print(direction)
        step_magnitudes = np.array(step_magnitudes)
        # print(step_magnitudes)
        step_magnitudes = step_magnitudes[np.where(step_magnitudes > 0)[0]]
        
        if len(step_magnitudes) == 0 : # Handling unbounded case
            print(f"The problem is unbounded. Can't find a optimal solution!")
            return None
        
        min_step_magnitude = np.min(step_magnitudes)
        # print(min_step_magnitude)
        
        z = z.astype(float)
        z += min_step_magnitude * direction
        
        bool_mask, A_active, A_inactive = find_tight_rows(A, z, b)
        epsilons = -1*np.linalg.inv(A_active).T
        epsilons = epsilons.dot(c)

    # Print the final optimal vertex and cost
    cost = np.dot(c.T, z)
    print(f"Optimal Vertex: z = {tuple(z.flatten())}; Cost = {np.round(cost[0], 2)}")

    return z


In [11]:
def main():
    file_name = 'asg_2_test_case.csv'
    problem_stmt = pd.read_csv(file_name, header=None)
    
    z = problem_stmt.iloc[0, :-1].values.reshape(-1, 1)
    c = problem_stmt.iloc[1, :-1].values
    b = problem_stmt.iloc[2:, -1:].values
    A = problem_stmt.iloc[2:, :-1].values
    
    # print(z.shape)
    # print(c.shape)
    # print(b.shape)
    # print(A.shape)
    # print("///")
    
    b = remove_degeneracy(b)
    
    Az = np.dot(A,z) 
    S =  (Az <= b)
    if False in S:
        print(f"\nThe Initial Point z: {tuple(self.z)} is not feasible ")
        return None

    
    initital_neighbour = initial_feasible_vertex(A, b, c, z, len(c))
    # print(initital_neighbour)
    
    optimal_vertex = find_optimal_vertex(A, b, c, initital_neighbour)
    
    # z_optimal, z_cost_all, z_all = vertex_to_vertex_assign1(A, b, z, c, len(c))
    
    #print(optimal_vertex)

In [12]:
main()
 
# Test input for above Output

# 0,20,0
# 1,3,0
# -1,1,20
# -2,1,50
# -1,0,0
# 0,-1,0

NameError: name 'remove_degeneracy' is not defined